IMPORT LIBRARIES


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
import os


In [5]:
# prompt: read csv file named age_gender.csv

import pandas as pd
df = pd.read_csv('age_gender.csv')


DATA LOADING AND PRECESSING


In [ ]:
# Define paths
dataset_path = 'df'
image_size = (224, 224)
batch_size = 32

# Data augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

# Load data with ImageDataGenerator
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',
    subset='validation'
)


MODEL BUILDING


In [ ]:
# Load the ResNet50 model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of ResNet50
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='linear')  # Regression for age prediction
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

model.summary()


MODEL TRAINING


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)


EVALUATION AND VISUALIZATION

In [ ]:
# Evaluate on the validation set
val_loss, val_mae = model.evaluate(validation_generator)
print(f'Validation MAE: {val_mae:.2f}')

# Plot training & validation loss and MAE
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['mean_absolute_error'], label='Training MAE')
plt.plot(history.history['val_mean_absolute_error'], label='Validation MAE')
plt.legend()
plt.title('Training and Validation MAE')

plt.show()


REAL TIME IESTING

In [ ]:
# Load the model
model = tf.keras.models.load_model('path/to/your/saved_model')

# Real-time age prediction
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the image
    img = cv2.resize(frame, image_size)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0

    # Predict age
    predicted_age = model.predict(img)
    predicted_age = int(predicted_age[0][0])

    # Display the result
    cv2.putText(frame, f'Predicted Age: {predicted_age}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Age Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


SAVE MODEL

In [ ]:
model.save('age_detection_model.h5')
